### Extract from OSM
The purpose of this script is to show how to go from a .osm.pbf file to a network format that GOSTnets will accept. 
Additional modifications in this script include: clipping the roads to an input polygon (here, rek2.shp), and slicing the roads DataFrame by a list of accepted road types, as tagged in OSM (see accepted_road_types list). 

In [1]:
import geopandas as gpd
import pandas as pd
import os, sys

In [2]:
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append("../")
import GOSTnets as gn

networkx version: 2.4 
matplotlib version: 3.1.2 
osmnx version: 0.11 


In [3]:
import importlib

Define filepaths. Note the amendments that need to be made to file paths!

In [33]:
pth = "./" # change this path to your working folder
fil = r'iceland-latest.osm.pbf' # download this file from geofabrik: http://download.geofabrik.de/europe/iceland.html. 

# be sure to place the .osm.pbf file in the 'tutorial data' folder. 

f = os.path.join(pth, 'tutorial_data', fil)

Using the loadOSM library, pass the filepath of the .osm.pbf to initiate a OSM_to_network class

In [34]:
#dir(gn)
f

'./tutorial_data/iceland-latest.osm.pbf'

In [35]:
from osgeo import ogr

In [36]:
dir(ogr)

['ALTER_ALL_FLAG',
 'ALTER_DEFAULT_FLAG',
 'ALTER_NAME_FLAG',
 'ALTER_NULLABLE_FLAG',
 'ALTER_TYPE_FLAG',
 'ALTER_WIDTH_PRECISION_FLAG',
 'ApproximateArcAngles',
 'BuildPolygonFromEdges',
 'CreateGeometryFromGML',
 'CreateGeometryFromJson',
 'CreateGeometryFromWkb',
 'CreateGeometryFromWkt',
 'DataSource',
 'DataSource_swigregister',
 'DontUseExceptions',
 'Driver',
 'Driver_swigregister',
 'F_VAL_ALL',
 'F_VAL_ALLOW_NULL_WHEN_DEFAULT',
 'F_VAL_GEOM_TYPE',
 'F_VAL_NULL',
 'F_VAL_WIDTH',
 'Feature',
 'FeatureDefn',
 'FeatureDefn_swigregister',
 'Feature_swigregister',
 'FieldDefn',
 'FieldDefn_swigregister',
 'ForceTo',
 'ForceToLineString',
 'ForceToMultiLineString',
 'ForceToMultiPoint',
 'ForceToMultiPolygon',
 'ForceToPolygon',
 'GT_Flatten',
 'GT_GetCollection',
 'GT_GetCurve',
 'GT_GetLinear',
 'GT_HasM',
 'GT_HasZ',
 'GT_IsCurve',
 'GT_IsNonLinear',
 'GT_IsSubClassOf',
 'GT_IsSurface',
 'GT_SetM',
 'GT_SetModifier',
 'GT_SetZ',
 'GeneralCmdLineProcessor',
 'GeomFieldDefn',
 'Geom

In [ ]:
iceland = gn.OSM_to_network(f)

This generates a property of the class called 'roads_raw'. This is a raw dataframe of the highway objects, extracted from the .osm.pbf. This is the starting point for our network.

In [ ]:
iceland.roads_raw.infra_type.value_counts()

We define a list of the types of roads from the above that we consider acceptable for our road network

In [ ]:
accepted_road_types = ['residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link']

We call the filerRoads method and pass it a list of acceptable road types

In [ ]:
iceland.filterRoads(acceptedRoads = accepted_road_types)

In [ ]:
iceland.roads_raw.infra_type.value_counts()

Here, we import a shapefile for our area of interest. I want to make a map of Reykjavik, Iceland. I clip the extent of the country file to just the area around the capital (rek2.shp) below using standard GeoPandas functions:

In [ ]:
shp = gpd.read_file(os.path.join(pth, 'tutorial_data', 'rek2.shp'))
shp = shp.to_crs({'init':'epsg:4326'})
shp_obj = shp.geometry.iloc[0]

I check to see everything lines up by running intersect and counting the True / False returns:

In [ ]:
iceland.roads_raw.geometry.intersects(shp_obj).value_counts()

Then, I do this properly by modifying the roads_raw DF object - I remove any roads that don't intersect the capital area

In [ ]:
iceland.roads_raw = iceland.roads_raw.loc[iceland.roads_raw.geometry.intersects(shp_obj) == True]

having narrowed the DataFrame to the roads we are interested in, we generate the RoadsGDF object

In [ ]:
iceland.generateRoadsGDF(verbose = False)

With initialReadIn(), we transform this to a graph object

In [ ]:
iceland.initialReadIn()

We save this graph object down to file using gn.save()

In [ ]:
gn.save(iceland.network,'Iceland_unclean',os.path.join(pth, 'tutorial_outputs'))

Move on to Step 2 to see how we clean up this rough network.